<a href="https://colab.research.google.com/github/PTIch06/YasynskyiColab/blob/main/Lab6_Yasynskyi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Лабораторна робота №6
##з дисципліни "Нейронні мережі"
###студента групи КН-31/2
###Ясинського Дениса
###Варіант №11



1. Виконати вирішення задачі класифікації для 3 класів з набору даних food101 з використанням різних моделей нейронних мереж:<br>
1) CNN модель з лабораторної роботи 4<br>
2) Resnet модель<br>
3) Efficientnet модель (моделі 1.1-1.3 з використанням оптимізатора Adam)<br>
4) Моделі 1.2,1.3 з використанням оптимізатора SGD.<br>
5) Моделі 1.2,1.3 отримані за допомогою tf.keras.applications та треновані з використанням fine-tuning (останні 10 шарів)<br>
2. Індекси класів визначити індивідуально за залежностями: i1=n-1,i2=n+29,i3=n+59 (де і1,і2,і3 - індекс класу (починаючи з 0) у відсортованому за алфавітом наборі даних, n - номер за списком групи.
3. Порівняти результати моделювання із використанням TensorBoard
4. Графік(и) порівняння результатів завантажити у форматі .svg та вставити у підсумковий файл поряд із та відповідними висновками

In [17]:
#Завантажимо файли
import zipfile

!wget https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip

zip_ref = zipfile.ZipFile("101_food_classes_10_percent.zip", "r")
zip_ref.extractall()
zip_ref.close()

--2022-05-24 20:05:17--  https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.213.128, 173.194.214.128, 173.194.215.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.213.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1625420029 (1.5G) [application/zip]
Saving to: ‘101_food_classes_10_percent.zip.1’

101_food_classes_10 100%[===================>]   1.51G   168MB/s    in 9.3s    

2022-05-24 20:05:27 (166 MB/s) - ‘101_food_classes_10_percent.zip.1’ saved [1625420029/1625420029]



In [22]:
#Виділимо класи за варіантом (10, 40, 70)
import pathlib
import numpy as np
import os

data_dir_train = pathlib.Path("101_food_classes_10_percent/train/") 
data_dir_test = pathlib.Path("101_food_classes_10_percent/test/")
num_food_images_train = len(os.listdir(data_dir_train))
print('Count of images are in a file:', num_food_images_train)

class_names = np.array(sorted([item.name for item in data_dir_train.glob('*')]))
new_class_names = [class_names[10], class_names[40],class_names[70]]
print('Our classes:', new_class_names)

Count of images are in a file: 101
Our classes: ['bruschetta', 'french_fries', 'pad_thai']


In [28]:
#Видалимо зайве
from pathlib import Path
import shutil

for name in class_names:
  if name != new_class_names[0] and name != new_class_names[1] and name != new_class_names[2]:
    dirpath = Path(data_dir_train) / name
    dirpath_test = Path(data_dir_test) / name
    if dirpath_test.exists() and dirpath_test.is_dir():
        shutil.rmtree(dirpath)
        shutil.rmtree(dirpath_test)
print('Test:', os.listdir(data_dir_test))
print('Train:', os.listdir(data_dir_train))

Test: ['french_fries', 'bruschetta', 'pad_thai']
Train: ['french_fries', 'bruschetta', 'pad_thai']


In [29]:
#Конвертуємо зображення на дані
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1/255.)
test_datagen = ImageDataGenerator(rescale=1/255.)

train_data = train_datagen.flow_from_directory(train_dir,
                                               target_size=(224, 224),
                                               batch_size=32,
                                               class_mode='categorical')

test_data = train_datagen.flow_from_directory(test_dir,
                                              target_size=(224, 224),
                                              batch_size=32,
                                              class_mode='categorical')

Found 750 images belonging to 3 classes.
Found 225 images belonging to 3 classes.


In [30]:
#Урізноманітнимо зображення
train_datagen_augmented = ImageDataGenerator(rescale=1/255.,
                                             rotation_range=20,
                                             width_shift_range=0.2,
                                             height_shift_range=0.2,
                                             zoom_range=0.2,
                                             horizontal_flip=True)

train_data_augmented = train_datagen_augmented.flow_from_directory(train_dir,
                                                                  target_size=(224, 224),
                                                                  batch_size=32,
                                                                  class_mode='categorical')

Found 750 images belonging to 3 classes.


Навчання

In [31]:
#CNN
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense

m_1 = Sequential([
  Conv2D(32, 3, activation='relu', input_shape=(224, 224, 3)),
  MaxPool2D(),
  Conv2D(32, 3, activation='relu'),
  MaxPool2D(),
  Flatten(),
  Dense(3, activation='softmax')
])

m_1.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

tb_callback_1 = tf.keras.callbacks.TensorBoard(log_dir='logs/cnn', histogram_freq=1)

h_1 = m_1.fit(train_data_augmented,
                          epochs=10,
                          steps_per_epoch=len(train_data_augmented),
                          validation_data=test_data,
                          validation_steps=len(test_data),
                          callbacks=[tb_callback_1])

Epoch 1/10
24/24 [==============================] - 54s 2s/step - loss: 1.2522 - accuracy: 0.3720 - val_loss: 1.0684 - val_accuracy: 0.3600
Epoch 2/10
24/24 [==============================] - 51s 2s/step - loss: 1.0448 - accuracy: 0.4400 - val_loss: 0.9964 - val_accuracy: 0.4844
Epoch 3/10
24/24 [==============================] - 53s 2s/step - loss: 0.9306 - accuracy: 0.5600 - val_loss: 0.8854 - val_accuracy: 0.5867
Epoch 4/10
24/24 [==============================] - 53s 2s/step - loss: 0.8891 - accuracy: 0.5907 - val_loss: 0.8699 - val_accuracy: 0.6222
Epoch 5/10
24/24 [==============================] - 52s 2s/step - loss: 0.8248 - accuracy: 0.6480 - val_loss: 0.8447 - val_accuracy: 0.6844
Epoch 6/10
24/24 [==============================] - 52s 2s/step - loss: 0.7908 - accuracy: 0.6547 - val_loss: 0.8079 - val_accuracy: 0.6889
Epoch 7/10
24/24 [==============================] - 57s 2s/step - loss: 0.7683 - accuracy: 0.6813 - val_loss: 0.8761 - val_accuracy: 0.6311
Epoch 8/10
24/24 [==

In [35]:
#Resnet 
import tensorflow as tf
import tensorflow_hub as hub


m_2 = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/imagenet/resnet_v1_101/feature_vector/5",
                   trainable=False),
    tf.keras.layers.Dense(3, activation='softmax')
])
m_2.build([None, 224, 224, 3])

tb_callback_2 = tf.keras.callbacks.TensorBoard(log_dir='logs/resnet(adam)notunnig', histogram_freq=1)

m_2.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

h_2 = m_2.fit(train_data_augmented,
                          epochs=10,
                          steps_per_epoch=len(train_data_augmented),
                          validation_data=test_data,
                          validation_steps=len(test_data),
                          callbacks=[tb_callback_2])

Epoch 1/10
24/24 [==============================] - 299s 12s/step - loss: 0.4095 - accuracy: 0.8467 - val_loss: 0.2662 - val_accuracy: 0.9200
Epoch 2/10
24/24 [==============================] - 267s 11s/step - loss: 0.1491 - accuracy: 0.9507 - val_loss: 0.2434 - val_accuracy: 0.9156
Epoch 3/10
24/24 [==============================] - 268s 11s/step - loss: 0.1277 - accuracy: 0.9573 - val_loss: 0.2179 - val_accuracy: 0.9156
Epoch 4/10
24/24 [==============================] - 295s 12s/step - loss: 0.1031 - accuracy: 0.9653 - val_loss: 0.2113 - val_accuracy: 0.9156
Epoch 5/10
24/24 [==============================] - 269s 11s/step - loss: 0.0786 - accuracy: 0.9760 - val_loss: 0.2066 - val_accuracy: 0.9333
Epoch 6/10
24/24 [==============================] - 293s 12s/step - loss: 0.0631 - accuracy: 0.9787 - val_loss: 0.2237 - val_accuracy: 0.9333
Epoch 7/10
24/24 [==============================] - 294s 12s/step - loss: 0.0596 - accuracy: 0.9760 - val_loss: 0.2076 - val_accuracy: 0.9289
Epoch 

In [42]:
#Efficientnet(Adam)
import tensorflow as tf
import tensorflow_hub as hub


m_3 = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/imagenet/resnet_v1_101/feature_vector/5",
                   trainable=False),
    tf.keras.layers.Dense(3, activation='softmax')
])
m_3.build([None, 224, 224, 3])

tb_callback_2 = tf.keras.callbacks.TensorBoard(log_dir='logs/resnet(adam)notunnig', histogram_freq=1)

m_3.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

h_3 = m_3.fit(train_data_augmented,
                          epochs=10,
                          steps_per_epoch=len(train_data_augmented),
                          validation_data=test_data,
                          validation_steps=len(test_data),
                          callbacks=[tb_callback_2])

Epoch 1/10
24/24 [==============================] - 301s 12s/step - loss: 0.8466 - accuracy: 0.6813 - val_loss: 0.3162 - val_accuracy: 0.8889
Epoch 2/10
24/24 [==============================] - 287s 12s/step - loss: 0.1867 - accuracy: 0.9427 - val_loss: 0.2620 - val_accuracy: 0.9022
Epoch 3/10
24/24 [==============================] - 261s 11s/step - loss: 0.1328 - accuracy: 0.9680 - val_loss: 0.2049 - val_accuracy: 0.9200
Epoch 4/10
24/24 [==============================] - 287s 12s/step - loss: 0.1141 - accuracy: 0.9667 - val_loss: 0.2168 - val_accuracy: 0.9111
Epoch 5/10
24/24 [==============================] - 289s 12s/step - loss: 0.1105 - accuracy: 0.9653 - val_loss: 0.1871 - val_accuracy: 0.9289
Epoch 6/10
24/24 [==============================] - 261s 11s/step - loss: 0.0862 - accuracy: 0.9747 - val_loss: 0.1905 - val_accuracy: 0.9289
Epoch 7/10
24/24 [==============================] - 261s 11s/step - loss: 0.0749 - accuracy: 0.9773 - val_loss: 0.2049 - val_accuracy: 0.9111
Epoch 

In [44]:
#Efficientnet SGD
m_4 = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1",
                   trainable=False),
    tf.keras.layers.Dense(3, activation='softmax')
])
m_4.build([None, 224, 224, 3])

tb_callback_4 = tf.keras.callbacks.TensorBoard(log_dir='logs/effnet(sgd)notunning', histogram_freq=1)

m_4.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.SGD(),
              metrics=["accuracy"])

h_4 = m_4.fit(train_data_augmented,
                          epochs=10,
                          steps_per_epoch=len(train_data_augmented),
                          validation_data=test_data,
                          validation_steps=len(test_data),
                          callbacks = [tb_callback_4])

Epoch 1/10
24/24 [==============================] - 74s 3s/step - loss: 0.6784 - accuracy: 0.7893 - val_loss: 0.4687 - val_accuracy: 0.9156
Epoch 2/10
24/24 [==============================] - 60s 3s/step - loss: 0.3622 - accuracy: 0.9467 - val_loss: 0.3335 - val_accuracy: 0.9244
Epoch 3/10
24/24 [==============================] - 69s 3s/step - loss: 0.2594 - accuracy: 0.9693 - val_loss: 0.2769 - val_accuracy: 0.9333
Epoch 4/10
24/24 [==============================] - 60s 3s/step - loss: 0.2081 - accuracy: 0.9653 - val_loss: 0.2430 - val_accuracy: 0.9378
Epoch 5/10
24/24 [==============================] - 60s 2s/step - loss: 0.1771 - accuracy: 0.9800 - val_loss: 0.2222 - val_accuracy: 0.9422
Epoch 6/10
24/24 [==============================] - 69s 3s/step - loss: 0.1623 - accuracy: 0.9773 - val_loss: 0.2070 - val_accuracy: 0.9467
Epoch 7/10
24/24 [==============================] - 68s 3s/step - loss: 0.1454 - accuracy: 0.9787 - val_loss: 0.1965 - val_accuracy: 0.9467
Epoch 8/10
24/24 [==

In [45]:
#Resnet SGD fine-tuning
import tensorflow as tf
efficientnet_model = tf.keras.applications.efficientnet.EfficientNetB0(classes=3, weights=None)
for layer in efficientnet_model.layers:
  layer.trainable = False;

for layer in efficientnet_model.layers[-10:]:
  layer.trainable = True;
tb_callback_5 = tf.keras.callbacks.TensorBoard(log_dir='logs/effnet(sgd)tunnig', histogram_freq=1)

efficientnet_model.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.SGD(),
              metrics=["accuracy"])

h_5 = efficientnet_model.fit(train_data_augmented,
                          epochs=10,
                          steps_per_epoch=len(train_data_augmented),
                          validation_data=test_data,
                          validation_steps=len(test_data),
                          callbacks=[tb_callback_5])

Epoch 1/10
24/24 [==============================] - 73s 3s/step - loss: 1.1026 - accuracy: 0.3187 - val_loss: 1.0989 - val_accuracy: 0.3333
Epoch 2/10
24/24 [==============================] - 74s 3s/step - loss: 1.1001 - accuracy: 0.3440 - val_loss: 1.0990 - val_accuracy: 0.3333
Epoch 3/10
24/24 [==============================] - 74s 3s/step - loss: 1.1015 - accuracy: 0.3333 - val_loss: 1.1009 - val_accuracy: 0.3333
Epoch 4/10
24/24 [==============================] - 75s 3s/step - loss: 1.1024 - accuracy: 0.3147 - val_loss: 1.0987 - val_accuracy: 0.3333
Epoch 5/10
24/24 [==============================] - 67s 3s/step - loss: 1.0997 - accuracy: 0.3427 - val_loss: 1.1005 - val_accuracy: 0.3333
Epoch 6/10
24/24 [==============================] - 67s 3s/step - loss: 1.1006 - accuracy: 0.3253 - val_loss: 1.1040 - val_accuracy: 0.3333
Epoch 7/10
24/24 [==============================] - 75s 3s/step - loss: 1.1021 - accuracy: 0.3093 - val_loss: 1.0987 - val_accuracy: 0.3333
Epoch 8/10
24/24 [==

In [46]:
#Efficientnet SGD fine-tuning
eff_model = tf.keras.applications.efficientnet.EfficientNetB0(classes=3, weights=None)
for layer in eff_model.layers:
  layer.trainable = False;

for layer in eff_model.layers[-10:]:
  layer.trainable = True;

tensorboard_6 = tf.keras.callbacks.TensorBoard(log_dir='logs/efficientnet(sgd)tunnig', histogram_freq=1,
                                               write_graph=True, write_images=False)

eff_model.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.SGD(),
              metrics=["accuracy"])

history_model_6 = eff_model.fit(train_data_augmented,
                          epochs=10,
                          steps_per_epoch=len(train_data_augmented),
                          validation_data=test_data,
                          validation_steps=len(test_data),
                          callbacks=[tensorboard_6])

Epoch 1/10
24/24 [==============================] - 82s 3s/step - loss: 1.1007 - accuracy: 0.3173 - val_loss: 1.0990 - val_accuracy: 0.3333
Epoch 2/10
24/24 [==============================] - 65s 3s/step - loss: 1.1019 - accuracy: 0.3213 - val_loss: 1.0994 - val_accuracy: 0.3333
Epoch 3/10
24/24 [==============================] - 64s 3s/step - loss: 1.1013 - accuracy: 0.3307 - val_loss: 1.0988 - val_accuracy: 0.3333
Epoch 4/10
24/24 [==============================] - 74s 3s/step - loss: 1.1023 - accuracy: 0.3227 - val_loss: 1.0992 - val_accuracy: 0.3333
Epoch 5/10
24/24 [==============================] - 73s 3s/step - loss: 1.1030 - accuracy: 0.2920 - val_loss: 1.0988 - val_accuracy: 0.3333
Epoch 6/10
24/24 [==============================] - 73s 3s/step - loss: 1.1012 - accuracy: 0.3147 - val_loss: 1.0987 - val_accuracy: 0.3333
Epoch 7/10
24/24 [==============================] - 73s 3s/step - loss: 1.1017 - accuracy: 0.3107 - val_loss: 1.0992 - val_accuracy: 0.3333
Epoch 8/10
24/24 [==